In [1]:
from Pass1 import Pass1
from Objpgm import Objpgm

In [21]:
'''
# PASS2
- 명령어 어셈블(연산자 코드 번역, 주소 조사)->(optab이용)
- BYTE, WORD 등 정의되는 데이터 값 생성->symtab이용
- 어셈블러 지시자(byte resw) 처리
- 한줄씩 읽으면서->어셈블러 리스트(LISFILE) 생성,목적 프로그램 생성
'''

class Pass2:
    def __init__(self,Pass1):
        self.Pass1=Pass1
        self.intfile=self.Pass1.intfile
        self.optab=self.Pass1.optab
        self.symtab=self.Pass1.symtab
        self.objpgm=Objpgm(self.Pass1.srcname)
        self.lisfile=open(self.Pass1.srcname+"_lisfile","w")
        self.MODE="DIRECT_ADDRESSING"
        
        self.pass2()
        
        
    def pass2(self):
        first=self.intfile.lines[0]
        startAddr=first['LOCCTR']
        label,opcode,operand=first['label'],first['opcode'],first['operand']
        flag=first['flag']
        objectcode=''
        tempTR=[]
        
        self.objpgm.recordList.append({'LOCCTR':startAddr,'label':label,'opcode':opcode,'operand':operand})
        if opcode=='start' or 'START':
            self.lisfile.write(startAddr+'\t'+objectcode+'\t'+label+'\t'+opcode+'\t'+operand+'\t'+flag+'\n')
            self.objpgm.writeHR(self.Pass1.progLength[2:])
        
        for line in self.intfile.lines[1:]:
            if opcode.upper()=="END":
                break
            LOCCTR=line['LOCCTR']
            label,opcode,operand=line['label'],line['opcode'].upper().replace("\n",""),line['operand']
            flag=line['flag']
            self.objpgm.recordList.append({'LOCCTR':LOCCTR,'label':label,'opcode':opcode,'operand':operand})
            
            self.objpgm.tempSTRAddr.append(format(int(LOCCTR,16),'x'))
            
            if ',x' in operand:
                self.MODE="INDEX_ADDRESSING"
                
            if line['LOCCTR'][0]!='.':                                #not commentline?
                if self.optab.findO(opcode):
                    if self.symtab.findL(operand.replace(',x','')):                    #if there is a symbol in operand field then
                        operAddr=self.symtab.sym[operand.replace(',x','')][0][2:]
                        if self.MODE=="INDEX_ADDRESSING":#operandNum이 2개 필요한 경우, stch, ldch 같은 경우
                            objectcode=hex(int(operAddr,16)+8).replace("0x","")+operAddr
                            self.MODE="DIRECT_ADDRESSING"
                    elif opcode=="RSUB":
                        operAddr='0000'
                    else:
                        #set error flag (undefined symbol)->리스트파일
                        operAddr='0000'      #else store 0 as operand address
                        self.lisfile.write('Error: undefined symbol+\n')
                    objectcode=self.optab.opt[opcode.upper()][0]+operAddr      #assemble the object code instruction(연산자 코드 번역 optab이용
                    tempTR.append(objectcode)
                elif opcode.upper() =='BYTE':   #convert constant to object code
                    if operand[0].upper()=="X":
                        tempTR.append(operand[2:len(operand)-1])
                    elif operand[0].upper()=="C":
                        strtmp=operand[2:len(operand)-1]
                        objectcode=""
                        temp=[]
                        tmpstr=''
                        for i in strtmp:
                            tmpstr+=format(ord(i),"x") #문자->아스키->16진수
                            objectcode+=format(ord(i),"x") #문자->아스키->16진수
                            if len(objectcode)%6==0:
                                temp.append(tmpstr)
                                tmpstr=''
                        temp.append(tmpstr)
                        tempTR.append(objectcode)
                        if len(objectcode)>6:
                            self.lisfile.write(LOCCTR+'\t'+temp[0]+'\t'+label+'\t'+opcode+'\t'+operand+'\t'+flag+'\n')
                            for i in range(len(temp)-1):
                                if temp[i+1]=='':
                                    break
                                self.lisfile.write('\t'+'\t'+temp[i+1]+'\n')
                            continue
                        
                elif opcode.upper() =='WORD':
                    ###convert constant to object code->optab이용
                    objectcode=format(int(operand),'X') #int to hex(4비트) to str
                    objectcode='0'*(6-len(objectcode))+objectcode
                    tempTR.append(objectcode)
                else:#RESW,RESB
                    objectcode="-"
                    tempTR.append(objectcode)
                    
                
            #write listing line
            self.lisfile.write(LOCCTR+'\t'+objectcode+'\t'+label+'\t'+opcode+'\t'+operand+'\t'+flag+'\n')
            #read next input line
        self.objpgm.writeTR(tempTR) #바보야 줄잘맞춰
        self.objpgm.writeER(startAddr)
        #write last Text record to object program
        #write End record to object program
        #write last listing line            

In [22]:
def main():
    srcfile=input("src파일이름을 입력해주세요: ")
    p2=Pass2(Pass1(srcfile))
    
main()

src파일이름을 입력해주세요: test5


In [ ]:
t